In [1]:
import re
import sys

import nltk
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline

# nvdlib has be installed (I suggest to use venv for that)
from nvdlib.nvd import NVD

# add the experimental project to the path
sys.path.append("../src")

from toolkit.transformers import Hook, NBClassifier, cross_validate, FeatureExtractor
from toolkit.preprocessing import NVDFeedPreprocessor, LabelPreprocessor
from toolkit import pipelines, utils

### Get the NVD Feed

In [2]:
Hook.clear_current_instances()

In [3]:
# get the nvd feed
feed = NVD.from_feeds(feed_names=[2018, 'recent'])
feed.update()

DATA = list(feed.cves())

In [4]:
pipeline = Pipeline(
    steps=[
        (
            'nvd_prep',
            NVDFeedPreprocessor(['cve_id', 'project', 'description'])
        ),
        (
            'label_prep',
            LabelPreprocessor(
                feed_attributes=['project', 'description'],
                output_attributes=['cve_id', 'project', 'description'],
                hook=Hook(key='label_hook', func=utils.find_)
            )
        )
    ]
)

series = pipeline.fit_transform(DATA)

In [5]:
df = pd.DataFrame(series)

# Grid search

When performing grid search, exhaustive iteration over all combinations of feature extractor hooks will be performed in order to find an optimal combination.

The values used for evaluation of those combinations will be:
- precision
- cross validation mean
- cross validation standard deviation

The cross validation will be evaluated using 10-Fold XV at train-test split ratio 1:5.

In [6]:
# grid-search
from itertools import combinations, product, compress

# x-val
from sklearn.model_selection import train_test_split, KFold

# feature hooks
from toolkit.transformers import feature_hooks

In [7]:
cve_dict = {cve.cve_id: cve for cve in DATA}

In [10]:
feature_hooks.vendor_product_match_hook.default_kwargs = {'cve_dict': cve_dict}

FEATURE_HOOKS = [
    feature_hooks.is_alnum_hook,
    feature_hooks.has_uppercase_hook,
    feature_hooks.ver_precedes_hook,
    feature_hooks.ver_follows_hook,
    feature_hooks.ver_pos_hook,
    feature_hooks.word_len_hook,
    feature_hooks.word_in_dict_hook,
    feature_hooks.vendor_product_match_hook
]

In [11]:
# Create prediciton filters
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))
def stopwords_filter(t):
    word, _ = t[0]
    return word.lower() not in STOPWORDS

def version_filter(t):
    word, _ = t[0]
    return word != '<VERSION>'

def num_tag_filter(t):
    _, tag = t[0]
    return tag != 'NUM'

prediction_filters = [stopwords_filter, version_filter, num_tag_filter]

In [12]:
def parallel_evaluation(mask):
    """Execute parallel evaluation of feature hooks by given mask."""
    global RUN_NUM
    
    featureset, labels = pipelines.extract_labeled_features(
        data=DATA,
        nvd_attributes=['cve_id', 'description'],
        nltk_feed_attributes=['description'],
        feature_hooks=list(compress(FEATURE_HOOKS, mask)),
        labeling_func=utils.find_
    )
    
    # split the data into train / test set
    X_train, X_test, y_train, y_test = train_test_split(
        featureset, labels,
        test_size = 0.8,  # split in the ratio 1:5 to show how well the model generalizes
        random_state=0
    )

    clf = NBClassifier().fit(X_train)
    
    # precision
    precision = clf.evaluate(X_test[:], y_test[:], sample=True, n=3, filter_hooks=prediction_filters)
    
    # cross validation
    xval_score = cross_validate(
        clf,
        X_train,
        y_train,
        shuffle=True,
        n=3,
        sample=True,
        filter_hooks=prediction_filters
    )
    xval_mean, xval_std = xval_score.mean, xval_score.std
    
#     print(f"Finished with precision: {precision} ; xval_mean: {xval_mean} ; xval_std: {xval_std}")
    
    return precision, xval_mean, xval_std

### Validation

In [13]:
import os
import sys

from multiprocessing import Pool


# binary mask for feature hooks
mask_list = list(product(*[[0, 1]] * len(FEATURE_HOOKS[:])))

In [14]:
# number of cores for processes
n_cores = os.cpu_count()

# prepare classifiers with multiprocessing pool
pool = Pool(processes=n_cores) 
proc = pool.map_async(func=parallel_evaluation,
                      iterable=mask_list)

pool.close()
pool.join()

In [15]:
eval_results = proc.get()

In [16]:
df_eval = pd.DataFrame(
    data=[(*acc, *mask) for acc, mask in zip(eval_results, mask_list)],
    columns=['precision', 'xval_mean', 'xval_std'] + [hook.key for hook in FEATURE_HOOKS]
)

df_eval.sort_values(by=['xval_mean', 'xval_std'], ascending=False)#.style.set_properties(**{'width':'10em'})

,precision,xval_mean,xval_std,is_alnum,has_uppercase,ver_precedes,ver_follows,ver_pos,word_len,word_in_dict,vendor_product_match
71,0.935937,0.937500,0.039528,0,1,0,0,0,1,1,1
87,0.935937,0.937500,0.039528,0,1,0,1,0,1,1,1
103,0.935937,0.937500,0.039528,0,1,1,0,0,1,1,1
119,0.935937,0.937500,0.039528,0,1,1,1,0,1,1,1
15,0.915625,0.931250,0.076291,0,0,0,0,1,1,1,1
31,0.915625,0.931250,0.076291,0,0,0,1,1,1,1,1
47,0.915625,0.931250,0.076291,0,0,1,0,1,1,1,1
63,0.915625,0.931250,0.076291,0,0,1,1,1,1,1,1
135,0.921875,0.931250,0.070986,1,0,0,0,0,1,1,1
151,0.921875,0.931250,0.070986,1,0,0,1,0,1,1,1


#### Examination of the best feature hook combination

In [32]:
featureset, labels = pipelines.extract_labeled_features(
    data=DATA,
    nvd_attributes=['cve_id', 'description'],
    nltk_feed_attributes=['description'],
    feature_hooks=list(compress(FEATURE_HOOKS, mask_list[119])),  # use the default
    labeling_func=utils.find_
)

# split the data into train / test set
X_train, X_test, y_train, y_test = train_test_split(
    featureset, labels,
    test_size = 0.2,  # split in the ratio 1:5 to show how well the model generalizes
    random_state=0
)

clf = NBClassifier().fit(X_train)

score = cross_validate(
    clf,
    X_train,
    y_train,
    shuffle=True,
    n=3,
    sample=True,
    filter_hooks=prediction_filters
)

score

Score(values=array([0.96875   , 0.9375    , 0.921875  , 0.984375  , 0.921875  ,
       0.953125  , 0.984375  , 0.953125  , 0.984375  , 0.95238095]), mean=0.9561755952380953, std=0.022975173652863903)

In [33]:
print("Cross validation accuracy: {:5.2f} (+/- {:5.3f}) %".format(score.mean * 100, score.std * 200))

Cross validation accuracy: 95.62 (+/- 4.595) %


In [34]:
candidate_arr = clf.fit_predict(X_test, n=3, sample=True, filter_hooks=prediction_filters)

incorrect_predictions = dict()
correct_predictions = dict()

index = 0
for candidates, label in zip(candidate_arr, y_test):
    pred = clf._valid_candidates(candidates, label)
    if pred:
        correct_predictions[index] = [c[0][0] for c in candidates]
    else:
        incorrect_predictions[index] = [c[0][0] for c in candidates]
    index += 1

In [35]:
print("Correctly predicted:", len(correct_predictions), "out of", X_test.shape[0])

Correctly predicted: 152 out of 160


In [36]:
# project the incorrect prediction into separate df
df_incorrect = df.loc[[*incorrect_predictions.keys()]]
df_incorrect['incorrect']  = pd.Series(incorrect_predictions)

# project the correct predictions into separate df
df_correct = df.loc[[*correct_predictions.keys()]]
df_correct['incorrect']  = pd.Series(correct_predictions)

### Missclassified

In [37]:
df_incorrect

,cve_id,project,description,label,incorrect
4,CVE-2018-0608,h2o,Buffer overflow in H2O version 2.2.4 and earli...,H2O,"[graphite2, libgraphite2, Segment.cpp]"
20,CVE-2018-1000060,sensu,"Sensu, Inc. Sensu Core version Before 1.2.0 & ...",Sensu,"[com.sun.net.ssl.HostnameVerifier, java.protoc..."
27,CVE-2018-1000076,rubygems,RubyGems version Ruby 2.2 series: 2.2.9 and ea...,RubyGems,"[HTML, crafted, gem]"
88,CVE-2018-1000547,corebos,coreBOS version 7.0 and earlier contains a Inc...,coreBOS,"[Versions, Affected, Sprockets.]"
90,CVE-2018-1000559,qutebrowser,qutebrowser version introduced in v0.11.0 (117...,qutebrowser,"[AKCMS, &, &]"
138,CVE-2018-10296,MiniCMS,MiniCMS V1.10 has XSS via the mc-admin/post-ed...,MiniCMS,"[stored, XSS]"
154,CVE-2018-10529,LibRaw,An issue was discovered in LibRaw 0.18.9. Ther...,LibRaw,"[Makone, System, Support]"
158,CVE-2018-10539,WavPack,An issue was discovered in WavPack 5.1.0 and e...,WavPack,"[OpenResty, Firewall, DISPUTED]"


## VISUALIZATION

In [38]:
# visualization tools - NOTE: These do not come in the requirements! (Hence the BONUS)
from plotly import graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [39]:
# initialize plotly
init_notebook_mode(connected=True)

In [40]:
# compute cross validation few more times to get more data
# NOTE: this might take a while to compute
x_val_scores = [
    cross_validate(clf, X_train, y_train, shuffle=True, n=3,
                   sample=True, filter_hooks=prediction_filters)
    for _ in range(10)
]

x_val_scores = np.vstack(x_val_scores)

In [41]:
multi_candidate_eval_scores = [
    clf.evaluate(X_test, y_test, n=i, sample=True)
    for i in range(1, 21)
]

eval_trace = go.Scatter(
    x=np.arange(start=1, stop=len(multi_candidate_eval_scores)),
    y=multi_candidate_eval_scores,
)

In [42]:
scores, mean, std = np.hstack(x_val_scores[:, 0]), x_val_scores[:, 1].mean(), x_val_scores[:, 2].std()

# subtract from 1 to get distane from the middle
scores = 1.0 - scores
mean = 1.0 - mean

# show 2x std, as this is stated above as well
std *= 2

rad = np.linspace(0, 360, num=(len(scores) + 1)) * (np.pi / 180.)
rad = rad[:-1]

x = np.cos(rad) * (scores)
y = np.sin(rad) * (scores)
    
data_labels = ["{:5.3f} %".format((1 - score) * 100) for score in scores]
score_trace = go.Scatter(
    x=x,
    y=y,
    mode='markers',
    hovertext=data_labels,
    hoverinfo='text',
)

In [43]:
cross_validation_layout = {
    'title': 'Cross validation bias-variance tradeoff',
    'xaxis': {
        'zeroline': False,
        'ticks': '',
        'showticklabels': False
        
    },
    'yaxis': {
        'zeroline': False,
        'scaleanchor': 'x',
        'scaleratio': 1,
        'ticks': '',
        'showticklabels': False
    },
    'shapes': [
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': -mean,
            'y0': -mean,
            'x1': mean,
            'y1': mean,
            'opacity': 0.1,
            'fillcolor': 'red',
            'line': {
                'color': 'red',
            }
        },
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': -mean - std,
            'y0': -mean - std,
            'x1': mean + std,
            'y1': mean + std,
            'opacity': 0.8,
            'line': {
                'color': 'orange',
                'dash': 'dashdot',
            }
        },
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': -mean + std,
            'y0': -mean + std,
            'x1': mean - std,
            'y1': mean - std,
            'opacity': 0.8,
            'line': {
                'color': 'orange',
                'dash': 'dashdot',
            }
        },
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': -0.001,
            'y0': -0.001,
            'x1': 0.001,
            'y1': 0.001,
            'opacity': 0.8,
            'fillcolor': 'red',
            'line': {
                'color': 'red',
            }
        },
    ]
}

In [44]:
# visualize cross validation accuracy
fig = {
    'data': [score_trace],
    'layout': cross_validation_layout
}

iplot(fig, show_link=False)

In [45]:
multi_candidate_eval_layout = go.Layout(
    title="Accuracy increase per number of candidates.",
    yaxis=dict(
        title='Accuracy',
        titlefont=dict(
            color='grey'
        )
    ),
    xaxis=dict(
        title='Candidates',
        titlefont=dict(
            color='grey'
        )
    ),
)

In [46]:
fig = go.Figure(data=[eval_trace], layout=multi_candidate_eval_layout)

iplot(fig, show_link=False)